### Pennylane Intro to QSVT

**Aryan Bawa**  

In this notebook, I follow Pennylane.ai's intro to QSVT tutorial [here](https://pennylane.ai/qml/demos/tutorial_intro_qsvt) and make some notes about the process. 

In [2]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt